In [1]:
# !pip install catboost pandas scikit-learn


In [1]:
import numpy as np
from catboost import CatBoostClassifier, Pool
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [2]:
train_comp = pd.read_csv("train.csv").drop("id", axis=1)
train_orig = pd.read_csv("trainn.csv")
# test_orig = pd.read_csv("testt.csv")
train_orig["prognosis"] = train_orig["prognosis"].apply(lambda x: x.replace(" ", "_"))
# test_orig["prognosis"] = test_orig["prognosis"].apply(lambda x: x.replace(" ", "_"))


In [3]:
test_full = pd.read_csv("test.csv")


In [4]:
train_full = pd.concat([train_comp, train_orig])

X = train_full.drop("prognosis", axis=1).to_numpy()
y = train_full["prognosis"].to_numpy()

In [6]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [5]:
params = {
    "min_data_in_leaf": 1,
    "depth": 5,
    "l2_leaf_reg": 1,
    "iterations": 1000,
    # "iterations": 883,
    "learning_rate": 0.01,
}
model = CatBoostClassifier(
    **params,
    loss_function="MultiClass",
    verbose=True,
    thread_count=-1,
    task_type="GPU"
    
    # eval_metric="AUC",
    # early_stopping_rounds=20,
    # eval_metric="PRAUC:use_weights=false"
)
# train the model
# model.fit(X, y)
model.fit(X,y)


0:	learn: 2.3872004	total: 6.27ms	remaining: 6.27s
1:	learn: 2.3765720	total: 10.8ms	remaining: 5.41s
2:	learn: 2.3654314	total: 10.8ms	remaining: 5.41s
3:	learn: 2.3547781	total: 15.3ms	remaining: 5.09s
4:	learn: 2.3440726	total: 20.9ms	remaining: 5.2s
5:	learn: 2.3349398	total: 25ms	remaining: 4.96s
6:	learn: 2.3254453	total: 31.2ms	remaining: 5.17s
7:	learn: 2.3158757	total: 36.7ms	remaining: 5.2s
8:	learn: 2.3071218	total: 41.5ms	remaining: 5.14s
9:	learn: 2.2975519	total: 45.6ms	remaining: 5.02s
10:	learn: 2.2881913	total: 51.2ms	remaining: 5.07s
11:	learn: 2.2787892	total: 55.5ms	remaining: 4.98s
12:	learn: 2.2698131	total: 59.6ms	remaining: 4.9s
13:	learn: 2.2619265	total: 64.8ms	remaining: 4.91s
14:	learn: 2.2535649	total: 64.8ms	remaining: 4.91s
15:	learn: 2.2452712	total: 64.8ms	remaining: 4.91s
16:	learn: 2.2367497	total: 70ms	remaining: 4.91s
17:	learn: 2.2289871	total: 74.4ms	remaining: 4.87s
18:	learn: 2.2221130	total: 78.5ms	remaining: 4.81s
19:	learn: 2.2149901	total: 8

In [6]:
ids = test_full["id"]
X_test = test_full.drop("id", axis=1).to_numpy()

In [7]:
predictions = model.predict_proba(X_test)
y_test = model.classes_[np.argsort(-predictions, axis=1)[:, :3]]
processed = pd.DataFrame(y_test).agg(" ".join, axis=1).values
answer = pd.DataFrame({"id": ids.values, "prognosis": processed})
answer.to_csv("final.csv", index=False)


In [15]:
model.save_model("model.cbm",
           format="cbm",
           export_parameters=None,
           pool=None)


In [18]:
model = CatBoostClassifier(
    **params,
    loss_function="MultiClass",
    verbose=True,
    thread_count=-1,
    task_type="GPU")

model.load_model("model.cbm")

In [19]:


predictions = model.predict_proba(X_test)
y_test = model.classes_[np.argsort(-predictions, axis=1)[:, :3]]
processed = pd.DataFrame(y_test).agg(" ".join, axis=1).values
answer = pd.DataFrame({"id": ids.values, "prognosis": processed})
answer.to_csv("final.csv", index=False)

In [20]:
answer


,id,prognosis
0,707,Rift_Valley_fever Tungiasis Japanese_encephalitis
1,708,Dengue Chikungunya Plague
2,709,West_Nile_fever Japanese_encephalitis Zika
3,710,Japanese_encephalitis Rift_Valley_fever Zika
4,711,Zika West_Nile_fever Malaria
...,...,...
298,1005,Zika Yellow_Fever West_Nile_fever
299,1006,Malaria Lyme_disease Plague
300,1007,Plague West_Nile_fever Lyme_disease
301,1008,Rift_Valley_fever West_Nile_fever Zika
